In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [12]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
#read data
#INSERT DATA HERE

In [5]:
# Load the data
#file_path = Path('')
#df = pd.read_csv(file_path, skiprows=1)[:-2]
#df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')

# Drop the null rows
#df = df.dropna()

#df.reset_index(inplace=True, drop=True)

#df.head()

In [6]:
# Create our features
#X = df.drop(columns='loan_status')
#X = pd.get_dummies(X)
# Create our target
#y = df.loc[:, target].copy()

In [7]:
#X.columns

In [8]:
#X.shape

In [9]:
#X.describe()

In [10]:
# Check the balance of our target values
#y[''].value_counts()

In [13]:
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Resample the training data with the RandomOversampler
#from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler(random_state=1)
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
#Counter(y_resampled)

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
#from imblearn.ensemble import BalancedRandomForestClassifier
#brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
#brf.fit(X_train, y_train)

In [15]:
# Calculated the balanced accuracy score
#y_pred = brf.predict(X_test)
#balanced_accuracy_score(y_test, y_pred)

In [16]:
# Display the confusion matrix
#confusion_matrix(y_test, y_pred)

In [17]:
# Print the imbalanced classification report
#print(classification_report_imbalanced(y_test, y_pred))

In [18]:
# List the features sorted in descending order by feature importance
#importances = brf.feature_importances_
#cols = X.columns
#feature_importances_df = pd.DataFrame({'feature':cols, 'importance': importances})
#feature_importances_df.head()
#feature_importances_df.sort_values('importance', ascending=False)

In [19]:
# Train the EasyEnsembleClassifier
#from imblearn.ensemble import EasyEnsembleClassifier
#eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
#eec.fit(X_train, y_train)

In [20]:
# Calculated the balanced accuracy score
#y_pred = eec.predict(X_test)
#balanced_accuracy_score(y_test, y_pred)

In [21]:
# Display the confusion matrix
#confusion_matrix(y_test, y_pred)

In [22]:
# Print the imbalanced classification report
#print(classification_report_imbalanced(y_test, y_pred))